# NOTE: We are not using whisper in our project because the WER is really poor

This is probably because fine tuning whisper to phoneme level needs to be done with more care which we were not able to figure out

# Install required packages

In [1]:
!pip install speechbrain
!pip install textgrid transformers librosa

# Import the following packages

In [2]:
import os
import sys
import torch
import logging
import speechbrain as sb
from hyperpyyaml import load_hyperpyyaml
import librosa
import csv
from google.colab import drive, files
from speechbrain.utils.distributed import run_on_main, if_main_process
from speechbrain.tokenizers.SentencePiece import SentencePiece
from speechbrain.utils.data_utils import undo_padding
from hyperpyyaml import load_hyperpyyaml
from pathlib import Path

# Mount google drive

In [3]:
# Mount drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# Define constants


In [4]:
logger = logging.getLogger(__name__)

#Change directory to folder in drive

folder_path = '/content/drive/MyDrive/CS5647_Project'
os.chdir(folder_path)
current_directory = os.getcwd()
print("Current Working Directory after change:", current_directory)

#import the mpd eval stats as it is in the same directory
from mpd_eval_v3 import MpdStats

Current Working Directory after change: /content/drive/MyDrive/CS5647_Project


# Defining the speech brain class

In [24]:
# Define training procedure
class ASR(sb.Brain):
    def compute_forward(self, batch, stage):
        """Forward computations from the waveform batches to the output probabilities."""
        batch = batch.to(self.device)
        wavs, wav_lens = batch.sig
        wavs, wav_lens = wavs.to(self.device), wav_lens.to(self.device)

        # Add augmentation if specified
        if stage == sb.Stage.TRAIN:
            if hasattr(self.hparams, "augmentation"):
                wavs = self.hparams.augmentation(wavs, wav_lens)

        # Forward pass

        # Encode with Whisper and then DNN
        feats = self.modules.whisper(wavs)
        x = self.modules.enc(feats)

        # Compute outputs
        logits = self.modules.ctc_lin(x)
        p_ctc = self.hparams.log_softmax(logits)

        return p_ctc, wav_lens

    def compute_objectives(self, predictions, batch, stage):
        "Given the network predictions and targets computed the NLL loss."

        p_ctc, wav_lens = predictions

        ids = batch.id
        targets, target_lens = batch.phn_encoded_target
        if stage != sb.Stage.TRAIN:
            canonicals, canonical_lens = batch.phn_encoded_canonical
            perceiveds, perceived_lens = batch.phn_encoded_perceived

        loss_ctc = self.hparams.ctc_cost(p_ctc, targets, wav_lens, target_lens)
        loss = loss_ctc

        # Record losses for posterity
        if stage != sb.Stage.TRAIN:
            # Note: sb.decoders.ctc_greedy_decode will also remove padded tokens
            # that is, it return a list of list with different lengths
            sequence = sb.decoders.ctc_greedy_decode(
                p_ctc, wav_lens, blank_id=self.hparams.blank_index
            )
            self.ctc_metrics.append(ids, p_ctc, targets, wav_lens, target_lens)

            self.per_metrics.append(
                ids=ids,
                predict=sequence,
                target=targets,
                predict_len=None,
                target_len=target_lens,
                ind2lab=self.label_encoder.decode_ndim,
            )
            self.mpd_metrics.append(
                ids=ids,
                predict=sequence,
                canonical=canonicals,
                perceived=perceiveds,
                predict_len=None,
                canonical_len=canonical_lens,
                perceived_len=perceived_lens,
                ind2lab=self.label_encoder.decode_ndim,
            )

        return loss

    def evaluate_batch(self, batch, stage):
        """Computations needed for validation/test batches"""
        predictions = self.compute_forward(batch, stage=stage)
        loss = self.compute_objectives(predictions, batch, stage=stage)
        return loss.detach()


    def on_stage_start(self, stage, epoch):
        """Gets called at the beginning of each epoch"""
        torch.cuda.empty_cache()
        self.ctc_metrics = self.hparams.ctc_stats()

        if stage != sb.Stage.TRAIN:
            self.per_metrics = self.hparams.per_stats()
            self.mpd_metrics = MpdStats()

    def on_stage_end(self, stage, stage_loss, epoch):
        """Gets called at the end of a epoch."""
        if stage == sb.Stage.TRAIN:
            self.train_loss = stage_loss
        else:
            per = self.per_metrics.summarize("error_rate")
            mpd_f1 = self.mpd_metrics.summarize("mpd_f1")

        if stage == sb.Stage.VALID:
            # old_lr_model, new_lr_model = self.hparams.lr_annealing_model(
            #     stage_loss
            # )
            # old_lr_whisper, new_lr_whisper = self.hparams.lr_annealing_whisper(
            #     stage_loss
            # )
            # sb.nnet.schedulers.update_learning_rate(
            #     self.adam_optimizer, new_lr_model
            # )
            # sb.nnet.schedulers.update_learning_rate(
            #     self.whisper_optimizer, new_lr_whisper
            # )

            self.hparams.train_logger.log_stats(
                stats_meta={
                    "epoch": epoch,
                    "lr_adam": self.adam_optimizer.param_groups[0]["lr"],
                    "lr_whisper": self.whisper_optimizer.param_groups[0]["lr"],
                },
                train_stats={"loss": self.train_loss},
                valid_stats={
                    "loss": stage_loss,
                    "ctc_loss": self.ctc_metrics.summarize("average"),
                    "PER": per,
                    "mpd_f1": mpd_f1
                },
            )
            self.checkpointer.save_and_keep_only(
                meta={"PER": per, "mpd_f1": mpd_f1}, min_keys=["PER"], max_keys=["mpd_f1"]
            )

        if stage == sb.Stage.TEST:
            self.hparams.train_logger.log_stats(
                stats_meta={"Epoch loaded": self.hparams.epoch_counter.current},
                test_stats={"loss": stage_loss, "PER": per, "mpd_f1": mpd_f1},
            )
            with open(self.hparams.wer_file, "w") as w:
                w.write("CTC loss stats:\n")
                self.ctc_metrics.write_stats(w)
                w.write("\nPER stats:\n")
                self.per_metrics.write_stats(w)
                print(
                    "CTC and PER stats written to file",
                    self.hparams.wer_file,
                )
            with open(self.hparams.mpd_file, "w") as m:
                m.write("MPD results and stats:\n")
                self.mpd_metrics.write_stats(m)
                print(
                    "MPD results and stats written to file",
                    self.hparams.mpd_file,
                )

    def fit_batch(self, batch):
        """Fit one batch, override to do multiple updates.

        The default implementation depends on a few methods being defined
        with a particular behavior:

        * ``compute_forward()``
        * ``compute_objectives()``

        Also depends on having optimizers passed at initialization.

        Arguments
        ---------
        batch : list of torch.Tensors
            Batch of data to use for training. Default implementation assumes
            this batch has two elements: inputs and targets.

        Returns
        -------
        detached loss
        """
        should_step = self.step % self.hparams.gradient_accumulation == 0

        # Managing automatic mixed precision
        if self.auto_mix_prec:
            self.whisper_optimizer.zero_grad()
            self.adam_optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                outputs = self.compute_forward(batch, sb.Stage.TRAIN)
            loss = self.compute_objectives(outputs, batch, sb.Stage.TRAIN)
            self.scaler.scale(loss / self.hparams.gradient_accumulation).backward()
            if should_step:
                self.scaler.unscale_(self.whisper_optimizer)
                self.scaler.unscale_(self.adam_optimizer)
                if self.check_gradients(loss):
                    if self.optimizer_step > self.hparams.warmup_steps:
                        # Here we added a warmup to the CTC encoder to make sure that
                        # it does not screw the whisper with too large gradients.
                        self.scaler.step(self.whisper_optimizer)
                    self.scaler.step(self.adam_optimizer)
                self.scaler.update()
                self.optimizer_step += 1
        else:
            outputs = self.compute_forward(batch, sb.Stage.TRAIN)

            loss = self.compute_objectives(outputs, batch, sb.Stage.TRAIN)
            # normalize the loss by gradient_accumulation step
            (loss / self.hparams.gradient_accumulation).backward()

            if self.step % self.hparams.gradient_accumulation == 0:
                # gradient clipping & early stop if loss is not fini
                if self.check_gradients(loss):
                    self.whisper_optimizer.step()
                    self.adam_optimizer.step()

                self.whisper_optimizer.zero_grad()
                self.adam_optimizer.zero_grad()

        return loss.detach().cpu()

    def init_optimizers(self):
        "Initializes the whisper optimizer and model optimizer"
        self.whisper_optimizer = self.hparams.whisper_opt_class(
            self.modules.whisper.parameters()
        )

        self.adam_optimizer = self.hparams.adam_opt_class(
            self.hparams.model.parameters()
        )

        if self.checkpointer is not None:
            self.checkpointer.add_recoverable(
                "whisper_opt", self.whisper_optimizer
            )
            self.checkpointer.add_recoverable("adam_opt", self.adam_optimizer)

    def on_fit_start(self):
        """Gets called at the beginning of ``fit()``, on multiple processes
        if ``distributed_count > 0`` and backend is ddp.

        Default implementation compiles the jit modules, initializes
        optimizers, and loads the latest checkpoint to resume training.
        """
        # Run this *after* starting all processes since jit modules cannot be
        # pickled.
        self._compile_jit()

        # Wrap modules with parallel backend after jit
        self._wrap_distributed()

        # Initialize optimizers after parameters are configured
        self.init_optimizers()

        # Load latest checkpoint to resume training if interrupted
        ## NOTE: make sure to use the "best" model to continual training
        ## so we set the `min_key` argument
        if self.checkpointer is not None:
            self.checkpointer.recover_if_possible(
                device=torch.device(self.device),
                min_key="PER"
            )



In [25]:
def dataio_prep(hparams):
    """This function prepares the datasets to be used in the brain class.
    It also defines the data processing pipeline through user-defined functions."""
    data_folder = hparams["data_folder_save"]

    train_data = sb.dataio.dataset.DynamicItemDataset.from_csv(
        csv_path=hparams["train_annotation"],
        replacements={"data_root": data_folder},
    )

    if hparams["sorting"] == "ascending":
        # we sort training data to speed up training and get better results.
        train_data = train_data.filtered_sorted(sort_key="duration")
        # when sorting do not shuffle in dataloader ! otherwise is pointless
        hparams["train_dataloader_opts"]["shuffle"] = False

    elif hparams["sorting"] == "descending":
        train_data = train_data.filtered_sorted(
            sort_key="duration", reverse=True
        )
        # when sorting do not shuffle in dataloader ! otherwise is pointless
        hparams["train_dataloader_opts"]["shuffle"] = False

    elif hparams["sorting"] == "random":
        pass

    else:
        raise NotImplementedError(
            "sorting must be random, ascending or descending"
        )

    valid_data = sb.dataio.dataset.DynamicItemDataset.from_csv(
        csv_path=hparams["valid_annotation"],
        replacements={"data_root": data_folder},
    )
    valid_data = valid_data.filtered_sorted(sort_key="duration")

    # test is separate
    test_data = sb.dataio.dataset.DynamicItemDataset.from_csv(
        csv_path=hparams["test_annotation"],
        replacements={"data_root": data_folder},
    )
    test_data = test_data.filtered_sorted(sort_key="duration")

    datasets = [train_data, valid_data, test_data]
    label_encoder = sb.dataio.encoder.CTCTextEncoder()

    # 2. Define audio pipeline:
    @sb.utils.data_pipeline.takes("wav")
    @sb.utils.data_pipeline.provides("sig")
    def audio_pipeline(wav):
        sig = sb.dataio.dataio.read_audio(wav)
        return sig

    sb.dataio.dataset.add_dynamic_item(datasets, audio_pipeline)

    # 3. Define text pipeline:
    @sb.utils.data_pipeline.takes("perceived_train_target")
    @sb.utils.data_pipeline.provides(
        "phn_list_target",
        "phn_encoded_list_target",
        "phn_encoded_target",
    )
    def text_pipeline_train(phn):
        phn_list = phn.strip().split()
        yield phn_list
        phn_encoded_list = label_encoder.encode_sequence(phn_list)
        yield phn_encoded_list
        phn_encoded = torch.LongTensor(phn_encoded_list)
        yield phn_encoded

    @sb.utils.data_pipeline.takes("perceived_train_target", "canonical_aligned", "perceived_aligned")
    @sb.utils.data_pipeline.provides(
        "phn_list_target",
        "phn_encoded_list_target",
        "phn_encoded_target",
        "phn_list_canonical",
        "phn_encoded_list_canonical",
        "phn_encoded_canonical",
        "phn_list_perceived",
        "phn_encoded_list_perceived",
        "phn_encoded_perceived",
    )
    def text_pipeline_test(target, canonical, perceived):
        phn_list_target = target.strip().split()
        yield phn_list_target
        phn_encoded_list_target = label_encoder.encode_sequence(phn_list_target)
        yield phn_encoded_list_target
        phn_encoded_target = torch.LongTensor(phn_encoded_list_target)
        yield phn_encoded_target
        phn_list_canonical = canonical.strip().split()
        yield phn_list_canonical
        phn_encoded_list_canonical = label_encoder.encode_sequence(phn_list_canonical)
        yield phn_encoded_list_canonical
        phn_encoded_canonical = torch.LongTensor(phn_encoded_list_canonical)
        yield phn_encoded_canonical
        phn_list_perceived = perceived.strip().split()
        yield phn_list_perceived
        phn_encoded_list_perceived = label_encoder.encode_sequence(phn_list_perceived)
        yield phn_encoded_list_perceived
        phn_encoded_perceived = torch.LongTensor(phn_encoded_list_perceived)
        yield phn_encoded_perceived

    sb.dataio.dataset.add_dynamic_item([train_data], text_pipeline_train)
    sb.dataio.dataset.add_dynamic_item([valid_data, test_data], text_pipeline_test)

    # 3. Fit encoder:
    # Load or compute the label encoder
    lab_enc_file = os.path.join(hparams["save_folder"], "label_encoder.txt")
    special_labels = {
        "blank_label": hparams["blank_index"],
    }
    label_encoder.load_or_create(
        path=lab_enc_file,
        from_didatasets=[train_data],
        output_key="phn_list_target",
        special_labels=special_labels,
        sequence_input=True,
    )

    # 4. Set output:
    sb.dataio.dataset.set_output_keys(
        [train_data],
        ["id", "sig", "phn_encoded_target"],
    )
    sb.dataio.dataset.set_output_keys(
        [valid_data, test_data],
        ["id", "sig", "phn_encoded_target", "phn_encoded_canonical", "phn_encoded_perceived"],
    )

    return train_data, valid_data, test_data, label_encoder

# Prepare dataloaders

In [27]:
hparams_file = '/content/drive/MyDrive/CS5647_Project/whisper/hparams/train.yaml'

# Load hyperparameters file with command-line overrides
with open(hparams_file) as fin:
    hparams = load_hyperpyyaml(fin)

# Create experiment directory
sb.create_experiment_directory(
    experiment_directory=hparams["output_folder"],
    hyperparams_to_save=hparams_file,
)

# Dataset IO prep: creating Dataset objects and proper encodings for phones
train_data, valid_data, test_data, label_encoder = dataio_prep(hparams)


speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: results/whisper_ctc
speechbrain.dataio.encoder - Load called, but CTCTextEncoder is not empty. Loaded data will overwrite everything. This is normal if there is e.g. an unk label defined at init.


In [28]:
hparams

{'seed': 3407,
 'output_folder': 'results/whisper_ctc',
 'wer_file': 'results/whisper_ctc/wer.txt',
 'mpd_file': 'results/whisper_ctc/mpd.txt',
 'save_folder': 'results/whisper_ctc/save',
 'train_log': 'results/whisper_ctc/train_log.txt',
 'whisper_hub': 'openai/whisper-base',
 'whisper_folder': 'results/whisper_ctc/save/whisper_checkpoint',
 'data_folder_save': './data',
 'train_annotation': './data/train_data.csv',
 'valid_annotation': './data/val_data.csv',
 'test_annotation': './data/test_data.csv',
 'ckpt_interval_minutes': 20,
 'number_of_epochs': 200,
 'warmup_steps': 400,
 'lr': 0.0004,
 'lr_whisper': 0.0004,
 'sorting': 'random',
 'auto_mix_prec': False,
 'sample_rate': 16000,
 'gradient_accumulation': 1.5,
 'token_type': 'char',
 'character_coverage': 1.0,
 'batch_size': 4,
 'num_workers': 4,
 'dnn_neurons': 1024,
 'freeze_whisper': False,
 'freeze_encoder': False,
 'whisper_output_dim': 512,
 'output_neurons': 42,
 'blank_index': 0,
 'train_dataloader_opts': {'batch_size': 4

# Instantiating the ASR SB trainer

In [29]:
# Trainer initialization
asr_brain = ASR(
    modules=hparams["modules"],
    hparams=hparams,
    checkpointer=hparams["checkpointer"],
    run_opts={"device":"cuda"}
)

asr_brain.label_encoder = label_encoder

speechbrain.core - Info: auto_mix_prec arg from hparam file is used
speechbrain.core - Info: ckpt_interval_minutes arg from hparam file is used
speechbrain.core - 74.5M trainable parameters in ASR


# Training the ASR

In [30]:
# Training
asr_brain.fit(
    asr_brain.hparams.epoch_counter,
    train_data,
    valid_data,
    train_loader_kwargs=hparams["train_dataloader_opts"],
    valid_loader_kwargs=hparams["valid_dataloader_opts"],
)

speechbrain.utils.checkpoints - Would load a checkpoint here, but none found yet.
speechbrain.utils.epoch_loop - Going into epoch 1


100%|██████████| 75/75 [00:03<00:00, 19.97it/s]


speechbrain.utils.train_logger - epoch: 1, lr_adam: 4.00e-04, lr_whisper: 4.00e-04 - train loss: 17.83 - valid loss: 3.56, valid ctc_loss: 3.56, valid PER: 1.00e+02, valid mpd_f1: 2.00e-01
speechbrain.utils.checkpoints - Saved an end-of-epoch checkpoint in results/whisper_ctc/save/CKPT+2023-11-21+11-07-03+00
speechbrain.utils.epoch_loop - Going into epoch 2


100%|██████████| 75/75 [00:03<00:00, 19.85it/s]


speechbrain.utils.train_logger - epoch: 2, lr_adam: 4.00e-04, lr_whisper: 4.00e-04 - train loss: 3.51 - valid loss: 3.61, valid ctc_loss: 3.61, valid PER: 1.00e+02, valid mpd_f1: 2.00e-01
speechbrain.utils.checkpoints - Saved an end-of-epoch checkpoint in results/whisper_ctc/save/CKPT+2023-11-21+11-07-28+00
speechbrain.utils.checkpoints - Deleted checkpoint in results/whisper_ctc/save/CKPT+2023-11-21+11-07-03+00
speechbrain.utils.epoch_loop - Going into epoch 3


...

speechbrain.utils.train_logger - epoch: 198, lr_adam: 4.00e-04, lr_whisper: 4.00e-04 - train loss: 2.24 - valid loss: 2.54, valid ctc_loss: 2.54, valid PER: 91.37, valid mpd_f1: 2.06e-01
speechbrain.utils.checkpoints - Saved an end-of-epoch checkpoint in results/whisper_ctc/save/CKPT+2023-11-21+12-37-58+00
speechbrain.utils.checkpoints - Deleted checkpoint in results/whisper_ctc/save/CKPT+2023-11-21+12-37-28+00
speechbrain.utils.epoch_loop - Going into epoch 199


100%|██████████| 75/75 [00:03<00:00, 19.59it/s]


speechbrain.utils.train_logger - epoch: 199, lr_adam: 4.00e-04, lr_whisper: 4.00e-04 - train loss: 2.23 - valid loss: 2.51, valid ctc_loss: 2.51, valid PER: 90.62, valid mpd_f1: 2.07e-01
speechbrain.utils.checkpoints - Saved an end-of-epoch checkpoint in results/whisper_ctc/save/CKPT+2023-11-21+12-38-28+00
speechbrain.utils.checkpoints - Deleted checkpoint in results/whisper_ctc/save/CKPT+2023-11-21+12-37-58+00
speechbrain.utils.epoch_loop - Going into epoch 200


100%|██████████| 75/75 [00:03<00:00, 19.68it/s]


speechbrain.utils.train_logger - epoch: 200, lr_adam: 4.00e-04, lr_whisper: 4.00e-04 - train loss: 2.25 - valid loss: 2.50, valid ctc_loss: 2.50, valid PER: 90.91, valid mpd_f1: 2.08e-01
speechbrain.utils.checkpoints - Saved an end-of-epoch checkpoint in results/whisper_ctc/save/CKPT+2023-11-21+12-39-13+00
speechbrain.utils.checkpoints - Deleted checkpoint in results/whisper_ctc/save/CKPT+2023-11-21+12-38-28+00


# Testing the model

In [31]:
asr_brain.evaluate(
    test_data,
    min_key="PER",
    test_loader_kwargs=hparams["test_dataloader_opts"],
)

speechbrain.utils.checkpoints - Loading a checkpoint from results/whisper_ctc/save/CKPT+2023-11-21+12-30-31+00


100%|██████████| 300/300 [03:08<00:00,  1.59it/s]

speechbrain.utils.train_logger - Epoch loaded: 183 - test loss: 2.58, test PER: 89.32, test mpd_f1: 2.02e-01
CTC and PER stats written to file results/whisper_ctc/wer.txt
MPD results and stats written to file results/whisper_ctc/mpd.txt


2.583135099013645